## 12.1 Dagger2 - Wstrzykiwanie zależności - Podstawy

Aplikacja posłuży nam do zapoznania się z ideą **dependency injection**, czyli wstrzykiwaniem zależności. Wykorzystamy tylko pojedynczą aktywność z polem `TextView` (bez fragmentów i nawigacji).

Rozpoczniemy od dodania odpowiednich zależności do projektu. Zależności możemy skopiować ze strony projektu na [github](https://github.com/google/dagger).

In [ ]:
dependencies {

    implementation 'com.google.dagger:dagger:2.44'
    annotationProcessor 'com.google.dagger:dagger-compiler:2.44'

    implementation 'com.google.dagger:dagger-android:2.44'
    implementation 'com.google.dagger:dagger-android-support:2.44'
    annotationProcessor 'com.google.dagger:dagger-android-processor:2.44'
}

Napiszmy kilka klas reprezentujących komputrer oraz jego składowe.

In [ ]:
public class Computer {}
public class Case {}
public class CPU {}
public class GPU {}
public class Motherboard {}
public class PowerSupply {}

Klasa `Computer` nie powinna odpowiadać za utworzenie zależności, więc przekazemy je do konstruktora głównego. Dodamy również jedną metodę.

In [ ]:
public class Computer {
    private Case computerCase;
    private GPU gpu;
    private CPU cpu;
    private Motherboard motherboard;
    private PowerSupply powerSupply;

    public Computer(
            Case computerCase,
            CPU cpu,
            GPU gpu,
            Motherboard motherboard,
            PowerSupply powerSupply){
        this.computerCase = computerCase;
        this.cpu = cpu;
        this.gpu = gpu;
        this.motherboard = motherboard;
        this.powerSupply = powerSupply;
    }
    
    public String work(){
        return "working";
    }
}

Przejdźmy do głównej aktynwości i dodajmy zmienną `Computer` oraz ustawmy text pola `TextView` w metodzie `onCreate`.

In [ ]:
public class MainActivity extends AppCompatActivity {
    
    private Computer computer;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        
        TextView textView = findViewById(R.id.textview);
        textView.setText(computer.work());
    }
}

Musimy jeszcze zainicjować obiekt `Computer`. Aby to zrobić musielibyśmy utworzyć szereg innych obiektów.

In [ ]:
public class MainActivity extends AppCompatActivity {

    private Computer computer;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

//         Case computerCase = new Case();
//         GPU gpu = new GPU();
//         CPU cpu = new CPU();
//         Motherboard motherboard = new Motherboard();
//         PowerSupply powerSupply = new PowerSupply();
        
//         computer = new Computer(computerCase, cpu, gpu, motherboard, powerSupply);

        TextView textView = findViewById(R.id.textview);
        textView.setText(computer.work());
    }
}

Ponieważ nie chcemy tego robić manualnie za każdym razem, wykorzystamy bibliotekę `Dagger2`.

Utwórzmy nowy interfejs o nazwie `computerComponent` - tutaj zostanie utworzony `DAG` ([Skierowany graf acykliczny](https://pl.wikipedia.org/wiki/Skierowany_graf_acykliczny)) w którym zostaną zawarte wszystkie informacje o zależnościach klasy `Computer` (od czego zależy klasa `Computer` oraz kolejność w jakich wszystkie zależności muszą zostać utworzone).

`@Component` tworzy i przechowuje obiekt, następnie dostarcza go do odbiorcy - często nazywany *injector*

In [ ]:
@Component
public interface ComputerComponent {
}

Zdefiniujmy jedną metodę `getComputer`

In [ ]:
@Component
public interface ComputerComponent {
    Computer getComputer();
}

`Dagger2` w czasie kompilacji zaimplementuje ten interfejs i doda automatycznie niezbędny kod. Kolejnym krokiem będzie odpowiednie oznaczenie wszystkich konstruktorów (**wstrzykiwanie przez konstruktor**) które musza zostać wykorzystane przy tworzeniu instancji klasy `Computer`.

In [ ]:
class Computer @Inject constructor (
    private val case: Case,
    private val gpu: GPU,
    private val cpu: CPU,
    private val motherboard: Motherboard,
    private val powerSupply: PowerSupply
        ) {
    fun work(): String{
        return "working"
    }
}

`Dagger2` musi jeszcze posiadać informację o sposobie zainicjowania wszystkich zależności.

In [ ]:
public class Case {
    @Inject
    public Case() {
    }
}

public class CPU {
    @Inject
    public CPU() {
    }
}

public class GPU {
    @Inject
    public GPU() {
    }
}

public class Motherboard {
    @Inject
    public Motherboard() {
    }
}

public class PowerSupply {
    @Inject
    public PowerSupply() {
    }
}

W klasie `MainActivity` nie możemy zastosować adnotacji `@Inject` na konstruktorze - instancję klasy `Computer` otrzymamy dzięki interfejsowi `ComputerComponent`. Ponieważ wykorzystaliśmy adnotację `@Component` mamy dostępną klasę `DaggerComputerComponent` wygenerowaną automatycznie, posiada ona metoda `create` dzięki której możemy stworzyć obiekt o typie interfejsu `ComputerComponent`

In [ ]:
ComputerComponent component = DaggerComputerComponent.create();

Następnie skorzystamy z metody `getComputer` zdefiniowanej w interfejsie `ComputerComponent`

In [ ]:
computer = component.getComputer();

Pełny kod `MainActivity`

In [ ]:
public class MainActivity extends AppCompatActivity {

    private Computer computer;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        ComputerComponent component = DaggerComputerComponent.create();
        computer = component.getComputer();

        TextView textView = findViewById(R.id.textview);
        textView.setText(computer.work());
    }
}